In [1]:
import sys

sys.path.append("/Users/thomas.cong/Downloads/ResearchCode")
import pandas as pd
from feature_calculation import build_biomarker_csv
import warnings

warnings.filterwarnings("ignore")

In [2]:
build_biomarker_csv.build_csv(
    "./Park_Celeb_Biomarkers.csv",
    audio_dir="/Users/thomas.cong/Downloads/ResearchCode/ParkCelebCode/Preprocessed Park Celeb Files",
    results_dir="./Park_Celeb_Biomarkers",
)

Processing audio files:   6%|▌         | 74/1289 [05:34<1:31:38,  4.53s/it]
Process SpawnPoolWorker-10:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/test/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/test/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/test/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/thomas.cong/Downloads/ResearchCode/feature_calculation/build_biomarker_csv.py", line 165, in process_audio_file
    audio_feats = audio_features.calculate_audio_features(preprocessed_path)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thomas.cong/Downloads/ResearchCode/feature_calculation/audio_features.py", line 487, in calculate_audio_features
    data["cpp"] = com

KeyboardInterrupt: 

In [13]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")
ParkCelebBiomarkers.drop_duplicates(inplace=True)


def get_yfd(filename):
    try:
        return float(filename.split("_")[-1])
    except:
        return None


ParkCelebBiomarkers["YFD"] = (
    ParkCelebBiomarkers.index.str.split("_").str[-1].map(get_yfd)
)
ParkCelebBiomarkers["Status"] = (
    ParkCelebBiomarkers.index.str.split("_").str[0].str.upper()
)
ParkCelebBiomarkers.to_csv("./Park_Celeb_Biomarkers_New.csv")

In [14]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")
ParkCelebBiomarkers["subjid"] = (
    ParkCelebBiomarkers.index.str.split("_").str[0:2].str.join("_")
)
ParkCelebBiomarkers.to_csv("./Park_Celeb_Biomarkers.csv")

In [3]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")

first_last_rows = []

for subjid, sub_df in ParkCelebBiomarkers.groupby("subjid"):
    # PRE-diagnosis window (-10 .. ‑6)
    pre = sub_df[sub_df["YFD"].between(-10, -6, inclusive="both")]
    if not pre.empty:
        first_idx = pre["YFD"].idxmin()          # closest to –10 (earliest)
        first_last_rows.append((first_idx, "First"))
    
    # POST-diagnosis window ( 6 .. 10)
    post = sub_df[sub_df["YFD"].between(6, 10, inclusive="both")]
    if not post.empty:
        last_idx  = post["YFD"].idxmax()         # closest to  10 (latest)
        first_last_rows.append((last_idx, "Last"))

# Build output DataFrame
indices, labels = zip(*first_last_rows)
first_last_df = ParkCelebBiomarkers.loc[list(indices)].copy()
first_last_df["First/Last"] = labels

first_last_df.to_csv("./Park_Celeb_Biomarkers_First_Last.csv")

In [5]:
first_last_df

,aavs,hull_area,speech_rate,articulation_rate,average_syllable_duration,avg_word_length,content_richness,mattr,phrase_patterns,sentence_length,...,shimmer_apq5,jitter_local,jitter_local_db,jitter_apq3,jitter_apq5,ppe,YFD,Status,subjid,First/Last
filename,,,,,,,,,,,,,,,,,,,,,
cn_02_7UuaSUdYtK8_10,121454.997176,6.957931e+05,2.200953,3.279603,0.304915,3.917578,0.557664,0.956816,4196,14.492795,...,0.076822,0.018069,0.000137,0.006959,0.008379,0.907747,10.0,CN,cn_02,Last
cn_03_4JpP9gG2Dsg_-7,105279.237770,5.416309e+05,3.954756,4.426402,0.225917,3.637838,0.583333,0.970339,13,13.214286,...,0.097011,0.031561,0.000294,0.014186,0.015357,0.908420,-7.0,CN,cn_03,First
cn_03_rwmHAR7IQwM_7,141430.095795,7.924498e+05,4.038075,4.788552,0.208831,4.091650,0.554656,0.969203,50,13.638889,...,0.063998,0.020765,0.000159,0.007967,0.010234,0.964745,7.0,CN,cn_03,Last
cn_04_xfMjQc0ltco_-10,131093.498248,7.736686e+05,3.691563,4.620853,0.216410,3.893333,0.538071,0.951858,67,22.222222,...,0.054994,0.020085,0.000162,0.008031,0.009163,0.895447,-10.0,CN,cn_04,First
cn_04_uN28hjq9YfM_10,142888.843728,1.028628e+06,3.474006,4.333457,0.230763,3.872093,0.583333,0.927000,55,18.428571,...,0.048449,0.027141,0.000209,0.010756,0.010781,0.977935,10.0,CN,cn_04,Last
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pd_34_p5q0eussR1U_-6,80648.232731,7.573815e+05,4.006978,4.653869,0.214875,4.394737,0.606218,0.964286,18,15.833333,...,0.076708,0.023882,0.000249,0.009005,0.011673,0.601180,-6.0,PD,pd_34,First
pd_36_qzhtgNKJIcE_10,105175.678837,7.090204e+05,3.787569,4.746092,0.210700,4.243029,0.566431,0.965970,723,19.725000,...,0.061374,0.023015,0.000152,0.011600,0.011237,0.733710,10.0,PD,pd_36,Last
pd_37_sd7qR8bB294_-8,105997.527370,8.148657e+05,4.163529,4.491376,0.222649,3.951724,0.571106,0.970433,66,13.593750,...,0.067458,0.017364,0.000152,0.006786,0.008349,0.992369,-8.0,PD,pd_37,First
